### Handling Unstructured Data with Python
**Description**: Extract structured data from unstructured text using Python.

**Steps**:
1. Load and analyze an unstructured text document.
2. Extract information using regex.

In [2]:
import re
import pandas as pd

# Step 1: Sample unstructured text
text = """
Customer: John Doe
Email: john.doe@example.com
Date: 2024-06-01
Amount: $1,200.50

Customer: Jane Smith
Email: jane_smith123@mail.com
Date: 2024-06-05
Amount: $980.00

Customer: Mike R.
Email: mike_r@example.org
Date: 2024-06-12
Amount: $2,450.75
"""

# Step 2: Extract data using regular expressions
pattern = r"Customer:\s*(.*?)\nEmail:\s*(.*?)\nDate:\s*(.*?)\nAmount:\s*\$([\d,]+\.\d{2})"

matches = re.findall(pattern, text)

# Step 3: Structure it into a DataFrame
df = pd.DataFrame(matches, columns=["Customer", "Email", "Date", "Amount"])

# Step 4: Convert amount to float
df['Amount'] = df['Amount'].str.replace(',', '').astype(float)

# Step 5: Display structured data
print(df)


     Customer                   Email        Date   Amount
0    John Doe    john.doe@example.com  2024-06-01  1200.50
1  Jane Smith  jane_smith123@mail.com  2024-06-05   980.00
2     Mike R.      mike_r@example.org  2024-06-12  2450.75
